Reading data from the package 

In [4]:
'''
import datapackage
import pandas as pd
from sklearn.model_selection import RepeatedKFold

data_url = 'https://datahub.io/machine-learning/tic-tac-toe/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        data_tic_tak = pd.read_csv(resource.descriptor['path'])
        #print (data)
'''
import pandas as pd
import numpy as np 

data_tic_tak  = pd.read_csv("./tic-tac-toe.csv")

In [5]:
#print(data.head())
attributes = data_tic_tak.columns.values
print(attributes)
attribute_values = data_tic_tak.iloc[:,0:9]
#print(attribute_values)
class_labels  = data_tic_tak.iloc[:,9]
#print(class_labels)

#decisionTree = DecisionTree(attribute_values, attributes, class_labels)
print(len(class_labels))

['top-left-square' 'top-middle-square' 'top-right-square'
 'middle-left-square' 'middle-middle-square' 'middle-right-square'
 'bottom-left-square' 'bottom-middle-square' 'bottom-right-square' 'Class']
958


In [6]:
#print(data[(data['top-middle-square'] == 'x')])
#print(data)
d1 = data_tic_tak['Class']
d2 = d1.loc[d1 == 'positive']
#print(d2)
attributes = data_tic_tak.iloc[:,0:9].columns.values
print(attributes)

['top-left-square' 'top-middle-square' 'top-right-square'
 'middle-left-square' 'middle-middle-square' 'middle-right-square'
 'bottom-left-square' 'bottom-middle-square' 'bottom-right-square']


In [55]:
import numpy as np
import math 
import operator

def CalEntropy(data, column_name):
    '''
    '''
    #print("***********", column_name)
    total_data_samples = len(data)
    #print("Total data samples", total_data_samples)

    col_feature_values = data[column_name]
    (vals, vals_count) = np.unique(col_feature_values, return_counts=True)
    #print("Values in the colum and there count", vals, vals_count)

    label_values = data.iloc[:,-1]
    class_labels  = np.unique(label_values)
    #print("Class Labels", class_labels)

    data_temp =  pd.concat([col_feature_values, label_values], axis=1)
    #print(data_temp.head())

    #class count for each feature variable
    #dictionary structure {'attribute value': [negetive sample, positive_sample]}
    label_count_dict = {}
    #print(data_temp.head())
    for x in vals:
        #print(column_name)
        d2 = data_temp.loc[data_temp[column_name] == x]
        d2 = d2.iloc[:,-1]
        (class_vals, class_vals_count) = np.unique(d2, return_counts=True)
        #print(class_vals, class_vals_count)  
        label_count_dict[x] = class_vals_count
    #print("$$$$$$$$ Label count dictionary is", label_count_dict)

    #entropy list stores the enropy for each feature variable 
    entropy_list = []
    #calculate entropy as -plog(p)
    for key in label_count_dict:
        if(len(label_count_dict[key]) > 1):
            neg_count = label_count_dict[key][0]
            pos_count = label_count_dict[key][1]
        #if ((pos_count != 0) or (neg_count != 0)):
            p1 = pos_count/(pos_count + neg_count)
            p2 = neg_count/(pos_count + neg_count)
            entropy = -((p1*math.log2(p1))+ (p2*math.log2(p2)))
            #print("Value of entropy is", entropy)
            entropy_list.append(entropy)
        else:
            entropy_list.append(0)
    #print("Entropy list", entropy_list)

    #Calculating the total entropy for the column 
    total_entropy = 0
    for x  in range(0, len(entropy_list)):
        total_entropy = total_entropy + (entropy_list[x]*(vals_count[x]/total_data_samples))
    #print("Total Entropy", total_entropy)

    return total_entropy

def CalInformationGain(data, attributes):
    '''
    '''
    total_data_samples = len(data)
    #print("Total data samples", total_data_samples)  
    label_values = data.iloc[:,-1]
    
    class_labels, label_counts  = np.unique(label_values, return_counts=True)
    #print("Class Labels", class_labels)
    #print("Label Count", label_counts)

    raw_entropy = 0
    for x  in range(0, len(label_counts)):
        p = label_counts[x]/total_data_samples
        raw_entropy = raw_entropy -(p*math.log2(p))
    #print("Raw entropy", raw_entropy)

    #information_gain_dict dictionary stores label of colum and IG corresponding to that 
    information_gain_dict = {}
    #attributes = data.iloc[:,0:9].columns.values
    for x in attributes:
        information_gain_dict[x] = raw_entropy - CalEntropy(data, x)

    #print("Dict of IG", (information_gain_dict))
    return information_gain_dict

def select_best_feature(information_gain_dict):
    sorted_information_gain_dict = sorted(information_gain_dict.items(), key=operator.itemgetter(1), reverse=True)
    #print(type(sorted_information_gain_dict))
    max_ig_feature = next(iter(sorted_information_gain_dict))
    ret = max_ig_feature[0]
    #print(ret)
    return ret 

class Node:
    def __init__(self, value=None, edge=None):
        self.value = value
        self.edge = edge #edge to this node (coming to this node)
        self.children = [] #list of Nodes
    def __str__(self):
        st = 'Node.value = '+str(self.value)+' edge_coming_to_me = '+str(self.edge)
        return st

#id3 tree will return the root node 
def id3(data, attributes, node):
    #print("Attributes for this iteration are", attributes)
    root = Node()
    root.edge = node.edge

    #check if all examples are positive or negative 
    class_labels = data.iloc[:,-1]
    labels, label_count = np.unique(class_labels, return_counts=True)
    #print(labels, label_count)

    if (len(labels) == 1):
        root.value = labels
        return root

    '''
    if (len(data) == 0):
        root.value = "END"
        return root
    '''
    '''
    for x in range(0,len(labels)):
        if label_count[x] == 0:
            root.value = labels[(x+1)%2]
            return root
    '''

    if attributes.size == 0:
        if(label_count[0]>=label_count[1]):
            root.value = labels[0]
        else:
            root.value = labels[1]
        return root
    else:
        information_gain_dict = CalInformationGain(data, attributes)
        A = select_best_feature(information_gain_dict)
        root.value = A
        root.children = []
        values_of_A = data[A]
        for e in values_of_A.unique():
            child_node = Node()
            #child_node.value = e
            child_node.edge = e
            #root.children.append(child_node)
            #dominant_label = data['Class'].value_counts()[0]
            data2 = data.loc[data[A] == e]
            #if len(data) == 0:
            #    child_node.edge = dominant_label
            if(len(data)>0):
                new_attributes = attributes[attributes != A]
                #TODO:
            #child_node.edge = id3(data, new_attributes)
            child_node = id3(data2, new_attributes, child_node)
            root.children.append(child_node)

    return root

#information_gain_dict = CalInformationGain(data, attributes)
#select_best_feature(information_gain_dict)
attributes = data_tic_tak.iloc[:,0:9].columns.values
#print(attributes)
#X = data.drop('Class', axis=1)
#y = data['Class']

data1 = pd.read_csv("./playtennis.csv")
attributes1 = data1.iloc[:,1:5].columns.values

temp_node = Node()
#root = id3(data_tic_tak, attributes, temp_node)
root = id3(data1, attributes1, temp_node)

In [11]:
def printNode(node):
    #print(node.edge)
    #print(node.value)
    st = 'Node.value = '+str(node.value)+' edge_coming_to_me = '+str(node.edge)
    print(st)

def printTree(node):
    if node:
        printNode(node)
        for c in node.children:
           printTree(c)
    else:
        return

printTree(root)

Node.value = Outlook edge_coming_to_me = None
Node.value = Humidity edge_coming_to_me = Sunny
Node.value = ['No'] edge_coming_to_me = High
Node.value = ['Yes'] edge_coming_to_me = Normal
Node.value = ['Yes'] edge_coming_to_me = Overcast
Node.value = Wind edge_coming_to_me = Rain
Node.value = ['Yes'] edge_coming_to_me = Weak
Node.value = ['No'] edge_coming_to_me = Strong


In [64]:
class_labels = data1.iloc[:,-1]
#class_labels = data_tic_tak.iloc[:,-1]
labels, label_count = np.unique(class_labels, return_counts=True)
def predict(tree, x_test, labels):
    root = tree
    predicted_val = ''
    v = root.value
    if v in labels:
        predicted_val = v[0]
        #print("predicted value = ", predicted_val)
        return predicted_val
    f = x_test[v]
    for child in root.children:
        if (f == child.edge):
            predicted_val = predict(child, x_test, labels)
    return predicted_val

y = predict(root, data1.iloc[1,:], labels)
print(y)

No


In [80]:
import numpy as np
from sklearn.model_selection import RepeatedKFold

def crossValidate(data1):
    temp_node = Node()
    class_labels = data1.iloc[:,-1]
    labels = np.unique(class_labels)
    y_predicted = []
    y_actual    = []
    y_accuracy  = []
    Accuracy = 0
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0
    rkf = RepeatedKFold(n_splits=10, n_repeats=2, random_state=2652124)
    for train_index, test_index in rkf.split(data1):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = data1.iloc[train_index, :], data1.iloc[test_index, :]
        #print(X_test)
        # train model 
        attributes = X_train.iloc[:,0:9].columns.values
        root = id3(X_train, attributes, temp_node)


        #validation 
        for _, test_row in data1.iterrows():
            #print(test_row)
            #print(type(test_row))
            actual = test_row.iloc[-1]
            predicted = predict(root, test_row, labels)
            if actual == predicted:
                if actual == 'positive':
                    true_pos += 1
                else:
                    true_neg += 1
            else:
                if predicted == 'positive':
                    false_pos += 1
                else:
                    false_neg += 1
            y_actual.append(actual)
            y_predicted.append(predicted)
            y_accuracy.append(int(actual == predicted))

    #print("y_actual: ", y_actual)
    #print("y_predicted: ", y_predicted)
    #print("y_accuracy: ", y_accuracy)
    Accuracy = 100 * np.count_nonzero(y_accuracy)/len(y_accuracy)
    Mean = np.mean(y_accuracy)
    Variance = np.var(y_accuracy)
    print("Accuracy: ", Accuracy, " %")
    print("Mean: ", Mean)
    print("Variance", Variance)
    print('\n\t\tConfusion Matrix')
    print("true_pos: ", true_pos, "\t\tfalse_neg: ", false_neg)
    print("false_pos: ", false_pos, "\t\ttrue_neg: ", true_neg)
    


crossValidate(data_tic_tak)

Accuracy:  98.54384133611691  %
Mean:  0.9854384133611691
Variance 0.014349546833390715

		Confusion Matrix
true_pos:  12374 		false_neg:  158
false_pos:  121 		true_neg:  6507


In [110]:
def addNoiseColumn(value_list):
    '''
    column_series = single value pandas series
    value_list = list of unique values in a column
    '''
    ind = np.random.randint(0, len(value_list))
    return value_list[ind]

def addNoise(data1, percent=.05, single_column=False):
    data_1 = data1.sample(frac=percent)
    print(data1)
    indices = data_1.index.tolist()
    print(indices)
    data_1 = data1.copy()
    value_list = {}
    if single_column:
        value_list = np.unique(data1)
        for i in indices:
            data_1[i] = addNoiseColumn(value_list)
    else:
        for c in data1:
            value_list[c] = np.unique(data1[c])
        for i in indices:
            for c in data_1:
                data_1[c][i] = addNoiseColumn(value_list[c])
    print(data_1)
    return data_1

data_1 = addNoise(data1.iloc[:, -1])


0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: Play-Tennis, dtype: object
[12]


KeyError: 'No'

In [106]:


print(data1)
percent = 0.5

print(data_1)


Day   Outlook Temperature Humidity    Wind Play-Tennis
0     1     Sunny         Hot     High    Weak          No
1     2     Sunny         Hot     High  Strong          No
2     3  Overcast         Hot     High    Weak         Yes
3     4      Rain        Mild     High    Weak         Yes
4     5      Rain        Cool   Normal    Weak         Yes
5     6      Rain        Cool   Normal  Strong          No
6     7  Overcast        Cool   Normal  Strong         Yes
7     8     Sunny        Mild     High    Weak          No
8     9     Sunny        Cool   Normal    Weak         Yes
9    10      Rain        Mild   Normal    Weak         Yes
10   11     Sunny        Mild   Normal  Strong         Yes
11   12  Overcast        Mild     High  Strong         Yes
12   13  Overcast         Hot   Normal    Weak         Yes
13   14      Rain        Mild     High  Strong          No
[11, 8, 10, 9, 0, 3, 2]
    Day   Outlook Temperature Humidity    Wind Play-Tennis
0     9     Sunny        Mild   Norm

In [57]:
import pprint
pp = pprint.PrettyPrinter()
pp.pprint(root)
print(root)

Node.value = middle-middle-square edge_coming_to_me = None


In [ ]:
def testTrainSplit (x, y, train_split = .9, fold):
    '''
    x attribute data (pandas)
    y class label column (pandas)
    '''
    len_data = len(x)
    test_idx = int(len_data*train_split)
    train_data = x.iloc[0:test_idx, :]


In [58]:
from collections import deque

def printTree(node_):
    if node_:
        roots = deque()
        roots.append(node_)
        while len(roots) > 0:
            root = roots.popleft()
            print(root.value)
            if root.children:
                for child in root.children:
                    print('({})'.format(child.value))
                    roots.append(child.edge)
            elif root.edge:
                print(root.edge)

printTree(root)

middle-middle-square
(top-left-square)
(top-left-square)
(top-left-square)


AttributeError: 'str' object has no attribute 'value'

In [22]:
class Node:
	def __init__(self, value=None, edge=None):
		self.value = value
		self.edge = edge #edge to this node (coming to this node)
		self.childs = [] #list of Nodes

'''
 for vi in values of bestAttr:
    // For example, Outlook has three values: Sunny, Overcast, and Rain
    insert value vi as branch of node
    create viRows with rows that only contains value vi
    if viRows is empty, then:
      this node branch ended by a leaf with value is dominant label in x
    else:
      newY = list of attributes y with bestAttr removed
      nextNode = next node connected by this branch
      nextNode = ID3(viRows, newY, label, nextNode)
  return node
'''
def id3(x, y, labels, node):
    '''
    x = training data without column names
    y = column names or attributes
    labels = labeled data (yes/no)
    node = is a class
    '''
    node = Node()
    c = np.unique(labels, count=True)
    if(len(c) == 1):
        node.value = c
        return node 
    if x.empty():
        node.value = #TODO
        return node
    
    information_gain_dict = CalInformationGain(data, attributes)
    best_attribute = select_best_feature(information_gain_dict)
    node.value = best_attribute
    best_attribute_vals = np.unique(x[best_attribute])

    node.childs = []
    for vi in best_attribute_vals:
        child = Node()
        child.edge = vi
        #node.childs.append(child)
    



1


In [ ]:
splitinfo = -(attr_x * np.ma.log2(attr_x))-(attr_o * np.ma.log2(attr_o))-(attr_b * np.ma.log2(attr_b))

In [ ]:
class tic_tac_node:

    def __init__(self, o_node, x_node, b_node, idxs, min_leaf=5):
        self.o_node = o_node 
        self.x_node = x_node
        self.b_node = b_node
        self.idxs = idxs 
        self.min_leaf = min_leaf
        self.row_count = len(idxs)
        self.col_count = x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()

In [ ]:
# Function: Calculates the Information Gain Ratio  
def info_gain_ratio(target, feature = [], uniques = []):
    entropy = 0
    denominator_1 = feature.count()
    data = pd.concat([pd.DataFrame(target.values.reshape((target.shape[0], 1))), feature], axis = 1)
    for entp in range(0, len(np.unique(target))):
        numerator_1 = data.iloc[:,0][(data.iloc[:,0] == np.unique(target)[entp])].count()
        if numerator_1 > 0:
            entropy = entropy - (numerator_1/denominator_1)* np.log2((numerator_1/denominator_1))
    info_gain = float(entropy)
    info_gain_r = 0
    intrinsic_v = 0
    for word in range(0, len(uniques)):
        denominator_2 = feature[(feature == uniques[word])].count()
        if denominator_2[0] > 0:
            intrinsic_v = intrinsic_v - (denominator_2/denominator_1)* np.log2((denominator_2/denominator_1))
        for lbl in range(0, len(np.unique(target))):
            numerator_2 = data.iloc[:,0][(data.iloc[:,0] == np.unique(target)[lbl]) & (data.iloc[:,1]  == uniques[word])].count()
            if numerator_2 > 0:
                info_gain = info_gain + (denominator_2/denominator_1)*(numerator_2/denominator_2)* np.log2((numerator_2/denominator_2))
    if intrinsic_v[0] > 0:
        info_gain_r = info_gain/intrinsic_v
    return float(info_gain_r)


In [ ]:
# x        -- training example 
# label    -- class label 
# features -- set of attributes
# samples  -- number of samples 

class decision_tree(object):
    def __init__(self, samples, features, labels)
    self.samples  = samples
    self.features = features
    self.labels   = labels
    self.

In [ ]:
def tic_tac_dt(data):
    node = tic_tac(True, )












class TicTac():
    def __init__(self, node_if_leaf, clssfiction, prnt, b_child, o_child,x_child, height):

        self.clssfiction = None
        self.split_atribute = None
        self.prnt = prnt
        self.b_child = None
        self.o_child = None
        self.x_child = None
        self.height = None
        self.node_if_leaf = True



















In [24]:
class Id3():

    def __init__(self):
        self.MAX = 7
        self.new_values = []
        self.target_attribute = None
        self.attributes = []
        print(self.MAX)

class DecisionTree():
    def __init__(self):
        self.id3 = Id3()

    def cal_entropy(self, max_int):
        pass

    def add_leaf_node(self):
        pass  

dt = DecisionTree()       

7


In [ ]:
def getEntropy(self, sampleIds):
		entropy = 0
		labelCount = [0] * len(self.labelCodes)
		for sid in sampleIds:
			labelCount[self.getLabelCodeId(sid)] += 1
		# print("-ge", labelCount)
		for lv in labelCount:
			# print(lv)
			if lv != 0:
				entropy += -lv/len(sampleIds) * math.log(lv/len(sampleIds), 2)
			else:
				entropy += 0
		return entropy

	def getInformationGain(self, sampleIds, attributeId):
		gain = self.getEntropy(sampleIds)
		attributeVals = []
		attributeValsCount = []
		attributeValsIds = []
		for sid in sampleIds:
			val = self.sample[sid][attributeId]
			if val not in attributeVals:
				attributeVals.append(val)
				attributeValsCount.append(0)
				attributeValsIds.append([])
			vid = attributeVals.index(val)
			attributeValsCount[vid] += 1
			attributeValsIds[vid].append(sid)
		# print("-gig", self.attributes[attributeId])
		for vc, vids in zip(attributeValsCount, attributeValsIds):
			# print("-gig", vids)
			gain -= vc/len(sampleIds) * self.getEntropy(vids)
		return gain


In [ ]:
f = open('tic-tac-toe.csv')